# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [5]:
def get_files(filepath):
  all_files = []
  for root, dirs, files in os.walk(filepath):
    files = glob.glob(os.path.join(root,'*.json'))
    for f in files :
      all_files.append(os.path.abspath(f))

  return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files('data/song_data')
print(len(song_files))
song_files[:3]

71


['/Users/harutaka/Desktop/udacity/data-engineer/projects/project1/data/song_data/A/A/A/TRAAAEF128F4273421.json',
 '/Users/harutaka/Desktop/udacity/data-engineer/projects/project1/data/song_data/A/A/A/TRAAARJ128F9320760.json',
 '/Users/harutaka/Desktop/udacity/data-engineer/projects/project1/data/song_data/A/A/A/TRAAAFD128F92F423A.json']

In [7]:
from pprint import pprint
import json

def read_json(filepath):
  with open(filepath, 'r') as f:
    return json.load(f)
    

filepath = song_files[1]
song = read_json(filepath)
pprint(song)

{'artist_id': 'AR8ZCNI1187B9A069B',
 'artist_latitude': None,
 'artist_location': '',
 'artist_longitude': None,
 'artist_name': 'Planet P Project',
 'duration': 269.81832,
 'num_songs': 1,
 'song_id': 'SOIAZJW12AB01853F1',
 'title': 'Pink World',
 'year': 1984}


In [8]:
song_df = pd.DataFrame([read_json(file) for file in song_files ])
song_df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR7G5I41187FB4CE6C,NaN,"London, England",NaN,Adam Ant,233.40363,1,SONHOTT12A8C13493C,Something Girls,1982
1,AR8ZCNI1187B9A069B,NaN,,NaN,Planet P Project,269.81832,1,SOIAZJW12AB01853F1,Pink World,1984
2,ARXR32B1187FB57099,NaN,,NaN,Gob,209.60608,1,SOFSOCN12A8C143F5D,Face the Ashes,2007
3,AR10USD1187B99F3F1,NaN,"Burlington, Ontario, Canada",NaN,Tweeterfriendly Music,189.57016,1,SOHKNRJ12A6701D1F8,Drop of Rain,0
4,ARGSJW91187B9B1D6B,35.21962,North Carolina,-80.01955,JennyAnyKind,218.77506,1,SOQHXMF12AB0182363,Young Boy Blues,0
5,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969
6,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0
7,ARKRRTF1187B9984DA,NaN,,NaN,Sonora Santanera,177.47546,1,SOXVLOJ12AB0189215,Amor De Cabaret,0
8,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0
9,ARKFYS91187B98E58F,NaN,,NaN,Jeff And Sheri Easter,267.70240,1,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
song_data = song_df[['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data

,song_id,title,artist_id,year,duration
0,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982,233.40363
1,SOIAZJW12AB01853F1,Pink World,AR8ZCNI1187B9A069B,1984,269.81832
2,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.60608
3,SOHKNRJ12A6701D1F8,Drop of Rain,AR10USD1187B99F3F1,0,189.57016
4,SOQHXMF12AB0182363,Young Boy Blues,ARGSJW91187B9B1D6B,0,218.77506
5,SOCIWDW12A8C13D406,Soul Deep,ARMJAGH1187FB546F3,1969,148.03546
6,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179
7,SOXVLOJ12AB0189215,Amor De Cabaret,ARKRRTF1187B9984DA,0,177.47546
8,SOUDSGM12AC9618304,Insatiable (Instrumental Version),ARNTLGG11E2835DDB9,0,266.39628
9,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),ARKFYS91187B98E58F,0,267.70240


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [10]:
cur.execute(song_table_drop)
cur.execute(song_table_create)
cur.executemany(song_table_insert, song_data.values.tolist())
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [11]:
artist_df = song_df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_df = artist_df.drop_duplicates('artist_id', keep='first').reset_index(drop=True)
artist_df

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR7G5I41187FB4CE6C,Adam Ant,"London, England",NaN,NaN
1,AR8ZCNI1187B9A069B,Planet P Project,,NaN,NaN
2,ARXR32B1187FB57099,Gob,,NaN,NaN
3,AR10USD1187B99F3F1,Tweeterfriendly Music,"Burlington, Ontario, Canada",NaN,NaN
4,ARGSJW91187B9B1D6B,JennyAnyKind,North Carolina,35.21962,-80.01955
5,ARMJAGH1187FB546F3,The Box Tops,"Memphis, TN",35.14968,-90.04892
6,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN
7,ARKRRTF1187B9984DA,Sonora Santanera,,NaN,NaN
8,ARNTLGG11E2835DDB9,Clp,,NaN,NaN
9,ARKFYS91187B98E58F,Jeff And Sheri Easter,,NaN,NaN


In [12]:
artist_df.shape

(69, 5)

In [13]:
artist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 5 columns):
artist_id           69 non-null object
artist_name         69 non-null object
artist_location     69 non-null object
artist_latitude     31 non-null float64
artist_longitude    31 non-null float64
dtypes: float64(2), object(3)
memory usage: 2.8+ KB


In [14]:
artist_df[artist_df['artist_name'].isnull()]

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [15]:
cur.execute(artist_table_drop)
cur.execute(artist_table_create)
cur.executemany(artist_table_insert, artist_df.values.tolist())
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
log_files = get_files('data/log_data')
log_files[:5]

['/Users/harutaka/Desktop/udacity/data-engineer/projects/project1/data/log_data/2018/11/2018-11-11-events.json',
 '/Users/harutaka/Desktop/udacity/data-engineer/projects/project1/data/log_data/2018/11/2018-11-23-events.json',
 '/Users/harutaka/Desktop/udacity/data-engineer/projects/project1/data/log_data/2018/11/2018-11-18-events.json',
 '/Users/harutaka/Desktop/udacity/data-engineer/projects/project1/data/log_data/2018/11/2018-11-04-events.json',
 '/Users/harutaka/Desktop/udacity/data-engineer/projects/project1/data/log_data/2018/11/2018-11-01-events.json']

In [17]:
def read_log_data(filepath):
  data = []
  with open(filepath) as f:
    for line in f:
        data.append(json.loads(line))
        
  return data

In [18]:
filepath = log_files[1]
read_log_data(filepath)

[{'artist': 'Great Lake Swimmers',
  'auth': 'Logged In',
  'firstName': 'Kevin',
  'gender': 'M',
  'itemInSession': 0,
  'lastName': 'Arellano',
  'length': 215.11791,
  'level': 'free',
  'location': 'Harrisburg-Carlisle, PA',
  'method': 'PUT',
  'page': 'NextSong',
  'registration': 1540006905796.0,
  'sessionId': 815,
  'song': 'Your Rocky Spine',
  'status': 200,
  'ts': 1542931645796,
  'userAgent': '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"',
  'userId': '66'},
 {'artist': 'Soziedad Alkoholika',
  'auth': 'Logged In',
  'firstName': 'Kevin',
  'gender': 'M',
  'itemInSession': 1,
  'lastName': 'Arellano',
  'length': 204.7473,
  'level': 'free',
  'location': 'Harrisburg-Carlisle, PA',
  'method': 'PUT',
  'page': 'NextSong',
  'registration': 1540006905796.0,
  'sessionId': 815,
  'song': 'Va Bien',
  'status': 200,
  'ts': 1542931860796,
  'userAgent': '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10

In [19]:
log_data = sum([read_log_data(file) for file in log_files], [])  # flatten list of list
log_data[:5]

[{'artist': 'Frumpies',
  'auth': 'Logged In',
  'firstName': 'Anabelle',
  'gender': 'F',
  'itemInSession': 0,
  'lastName': 'Simpson',
  'length': 134.47791,
  'level': 'free',
  'location': 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD',
  'method': 'PUT',
  'page': 'NextSong',
  'registration': 1541044398796.0,
  'sessionId': 455,
  'song': 'Fuck Kitty',
  'status': 200,
  'ts': 1541903636796,
  'userAgent': '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"',
  'userId': '69'},
 {'artist': 'Kenny G with Peabo Bryson',
  'auth': 'Logged In',
  'firstName': 'Anabelle',
  'gender': 'F',
  'itemInSession': 1,
  'lastName': 'Simpson',
  'length': 264.75057,
  'level': 'free',
  'location': 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD',
  'method': 'PUT',
  'page': 'NextSong',
  'registration': 1541044398796.0,
  'sessionId': 455,
  'song': 'By The Time This Night Is Over',
  'status': 200,
  'ts': 1541903770796,
 

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [20]:
log_df = pd.DataFrame(log_data)
log_df['datetime'] = pd.to_datetime(log_df['ts'], unit='ms')
log_df = log_df[log_df['page'] == 'NextSong']
log_df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,datetime
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 02:33:56.796
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 02:36:10.796
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69,2018-11-11 02:40:34.796
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32,2018-11-11 04:36:13.796
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75,2018-11-11 04:36:46.796
6,Gary Allan,Logged In,Ryann,F,0,Smith,259.83955,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,The One,200,1541930188796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-11 09:56:28.796
7,Miracle Fortress,Logged In,Ryann,F,1,Smith,200.90730,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,Five Roses,200,1541930447796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-11 10:00:47.796
8,Don Omar,Logged In,Ryann,F,2,Smith,261.35465,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,Cuentale,200,1541930647796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-11 10:04:07.796
9,Jay-Z,Logged In,Ryann,F,3,Smith,212.27057,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,D'Evils,200,1541930908796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-11 10:08:28.796
10,Red Hot Chili Peppers,Logged In,Ryann,F,4,Smith,231.33995,free,"Palestine, TX",PUT,NextSong,1.540685e+12,328,Easily (Album Version),200,1541931120796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,92,2018-11-11 10:12:00.796


In [21]:
time_data = [
  log_df['ts'],
  log_df['datetime'].dt.hour,
  log_df['datetime'].dt.day,
  log_df['datetime'].dt.weekofyear,
  log_df['datetime'].dt.month,
  log_df['datetime'].dt.year,
  log_df['datetime'].dt.dayofweek,
]

column_labels = [
  'timestamp',
  'hour',
  'day',
  'week_of_year',
  'month',
  'year',
  'weekday',
]


time_df = pd.concat(time_data, keys=column_labels, axis=1)
time_df.head()

,timestamp,hour,day,week_of_year,month,year,weekday
0,1541903636796,2,11,45,11,2018,6
1,1541903770796,2,11,45,11,2018,6
2,1541904034796,2,11,45,11,2018,6
4,1541910973796,4,11,45,11,2018,6
5,1541911006796,4,11,45,11,2018,6


In [22]:
# t = 
# t.head()

In [23]:
# time_data = ()
# column_labels = ()

In [24]:
# time_df = 
# time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [25]:
cur.execute(time_table_drop)
cur.execute(time_table_create)
for i, row in time_df.iterrows():
  cur.execute(time_table_insert, row.values.tolist())
  conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [26]:
user_df = log_df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df = user_df.drop_duplicates()
user_df = user_df[~user_df['userId'].eq('')]  # remove rows with empty user_id
user_df.head()

,userId,firstName,lastName,gender,level
0,69,Anabelle,Simpson,F,free
4,32,Lily,Burns,F,free
5,75,Joseph,Gutierrez,M,free
6,92,Ryann,Smith,F,free
12,49,Chloe,Cuevas,F,free


In [27]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104 entries, 0 to 7707
Data columns (total 5 columns):
userId       104 non-null object
firstName    104 non-null object
lastName     104 non-null object
gender       104 non-null object
level        104 non-null object
dtypes: object(5)
memory usage: 4.9+ KB


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [28]:
cur.execute(user_table_drop)
cur.execute(user_table_create)
for i, row in user_df.iterrows():
  cur.execute(user_table_insert, row)
  conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [29]:
from sql_queries import *

In [30]:
cur.execute(songplay_table_drop)
cur.execute(songplay_table_create)
for index, row in log_df.iterrows():
  # get songid and artistid from song and artist tables
  values = (row.song, row.artist, row.length)
  cur.execute(song_select, values)
  results = cur.fetchone()
  
  if results:
    songid, artistid = results
  else:
    songid, artistid = None, None
    

  # insert songplay record
  songplay_data = (
    row.ts,
    row.userId,
    row.level,
    songid,
    artistid,
    row.sessionId,
    row.location,
    row.userAgent,
  )
  
  cur.execute(songplay_table_insert, songplay_data)
  conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [31]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.